# Walking
The purpose of this notebook is to to refine a policy that has learned to move by trotting around into a policy that wlaks properly.

As usual the directory will contain all the necessary for the execution of this notebook.

In [1]:
import time
import numpy as np
from stable_baselines3 import PPO

import sys
import os

# Start from the current working directory (where notebook is)
cwd = os.getcwd()

# Go two levels up (to the "grandparent")
grandparent_dir = os.path.abspath(os.path.join(cwd, "..", ".."))

# Add to sys.path if not already there
if grandparent_dir not in sys.path:
    sys.path.insert(0, grandparent_dir)

from SpotmicroEnv import SpotmicroEnv
from reward_function import reward_function, RewardState

pybullet build time: Apr  4 2025 18:56:19


#### Training

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.logger import configure

# ========= CONFIG ==========
TOTAL_STEPS = 16_000_000
run = "walk"
base="trot"

log_dir = f"./logs/{run}"

def clipped_linear_schedule(initial_value, min_value=1e-5):
    def schedule(progress_remaining):
        return max(progress_remaining * initial_value, min_value)
    return schedule

checkpoint_callback = CheckpointCallback(
    save_freq=TOTAL_STEPS // 16,
    save_path=f"{run}_checkpoints",
    name_prefix=f"ppo_{run}"
)

# ========= ENV ==========
env = SpotmicroEnv(
    use_gui=False,
    reward_fn=reward_function, 
    reward_state=RewardState(),
    src_save_file=f"{base}.pkl",
    dest_save_file=f"{run}.pkl"
)
check_env(env, warn=True)

# ========= MODEL ==========
model = PPO.load(f"ppo_{base}", device = 'cpu')
model.set_env(env)
model.tensorboard_log = log_dir

# Custom logger: ONLY csv + tensorboard (no stdout table)
new_logger = configure(log_dir, ["csv", "tensorboard"])
model.set_logger(new_logger)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

In [ ]:
model.learn(
    total_timesteps=TOTAL_STEPS,
    reset_num_timesteps=False,
    callback=checkpoint_callback
)
model.save(f"ppo_{run}")
env.close()

# Evaluation

In [ ]:
policy = "walk"

env = SpotmicroEnv(
    use_gui=True, 
    reward_fn=reward_function,
    reward_state=RewardState(),
    #src_save_file=f"{policy}.pkl"
    )
obs, _ = env.reset()

# === Load model ===
#model = PPO.load(f"ppo_{policy}")
model = PPO.load(f"{policy}_checkpoints/ppo_{policy}_23002176_steps", device = 'cpu')

# === Run rollout ===
for _ in range(3001):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        print("Terminated")
        env.plot_reward_components()
        obs, _ = env.reset()
    time.sleep(1/60)

env.close()

ridges coefficient: 0.25
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=AMD
GL_RENDERER=AMD Radeon Graphics (radeonsi, renoir, LLVM 19.1.1, DRM 3.59, 6.11.0-21-generic)
GL_VERSION=4.6 (Core Profile) Mesa 24.2.8-1ubuntu1~24.04.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 24.2.8-1ubuntu1~24.04.1
Vendor = AMD
Renderer = AMD Radeon Graphics (radeonsi, renoir, LLVM 19.1.1, DRM 3.59, 6.11.0-21-generic)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = AMD
ven = AMD
b3Printf: b3Warning[examples/Importers/Im

# Results:
- n1 was an utter disaster, the gait moved alongisde the ridges (and so perpendicularly to the target direction). Maybe decrease forward reward AND target velocity